In [1]:
from functions import *
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from math import *

In [2]:
def Calibrate(afpDate,firstNMRDate,secondNMRDate,percent_per_khz,epicsData):
    import re
    import numpy as np
    
    trash, fShift, bShift=FindEPRShifts(afpDate)
    fit=np.load('NMRDataFits/Fringe.npy')
   
    firstNMR=np.where(fit[0].astype(int)==firstNMRDate)[0][0]
    secondNMR=np.where(fit[0].astype(int)==secondNMRDate)[0][0]
    #Get percent by multiplying front and back shift by %/khz
    fPer=fShift*percent_per_khz   
    bPer=bShift*percent_per_khz 
    
    f_pc_const=fPer/fit[1][firstNMR]
    b_pc_const=bPer/fit[1][secondNMR]
    
    f_us_const=fPer/fit[2][firstNMR]
    b_us_const=bPer/fit[2][secondNMR]
    
    f_ds_const=fPer/fit[3][firstNMR]
    b_ds_const=bPer/fit[3][secondNMR]
    
    pc_const=np.round((f_pc_const+b_pc_const)/2,4)
    us_const=np.round((f_us_const+b_us_const)/2,4)
    ds_const=np.round((f_ds_const+b_ds_const)/2,4)
    print(' Format of output is: EPR date, pc_const, us_const, ds_const')
    
    try:
        epicsFirst1=str(firstNMRDate)[:8]
        epicsFirst2=str(firstNMRDate)[8:]
        epicsFirst=epicsFirst1+'_'+epicsFirst2

        print(epicsFirst)
        epicsSecond1=str(secondNMRDate)[:8]
        epicsSecond2=str(secondNMRDate)[8:]
        epicsSecond=epicsSecond1+'_'+epicsSecond2

        firstTemp=np.empty(0)
        secondTemp=np.empty(0)
        for i in epicsData:   
            #print(i)
            firstTemp=np.append(firstTemp,np.round(GrabClosest(epicsFirst,i),4))
            secondTemp=np.append(secondTemp,np.round(GrabClosest(epicsSecond,i),4))
    except:
         yay=0
    return int(re.search(r'\d+', afpDate).group()), pc_const,us_const,ds_const,firstTemp,secondTemp
    
    

def FindEPRShifts(afpDate):
    file=np.loadtxt(afpDate)
    data=np.transpose(file)
    first=list(ordered_cluster(data[1], 10))
    second=Refine(first)
    return second


def GrabClosest(date,data):
    from datetime import datetime, timedelta
    import os
    import numpy as np
    goUp=False
    goDown=False
    found=False
   
    #get an initial index guess
    end=datetime.strptime(data[0][-1:][0],"%Y%m%d_%H%M%S")-datetime.strptime(data[0][0],"%Y%m%d_%H%M%S")
   
    
    valueNorm=datetime.strptime(date,"%Y%m%d_%H%M%S")-datetime.strptime(data[0][0],"%Y%m%d_%H%M%S")
  
    ratio=valueNorm.total_seconds()/end.total_seconds()

    index=int(ratio*len(data[0])-1)
    
    
    while found==False:
        baseline=abs(datetime.strptime(date,"%Y%m%d_%H%M%S")-datetime.strptime(data[0][index],"%Y%m%d_%H%M%S")).total_seconds()
        below=abs(datetime.strptime(date,"%Y%m%d_%H%M%S")-datetime.strptime(data[0][index-1],"%Y%m%d_%H%M%S")).total_seconds()
        above=abs(datetime.strptime(date,"%Y%m%d_%H%M%S")-datetime.strptime(data[0][index+1],"%Y%m%d_%H%M%S")).total_seconds()
        
        goDown=False
        goUp=False
        if below<baseline:
            goDown=True
            index=index-1
            
        if above<baseline:
            goUp=True
            index=index+1
            
        if goDown ==False and goUp==False:
            found=True
    
    ydat=data[1].astype(float)
   
    return(ydat[index])     

In [3]:
tempData=LoadEpics()
tc1,tc2,tc3,tc4,pc1,pc2,tt1,tt2=tempData

Format: tc1,tc2,tc3,tc4,pc1,pc2,tt1,tt2


In [6]:
#Fringe Calibration 1 with new settings
fringe_calibration1=Calibrate('EPRData/01250837.dat',20230125083654,20230125083905,.61886,tempData)

 Front shift is:  63.81 kHz 
 Back shift is:  60.58 kHz
 Format of output is: EPR date, pc_const, us_const, ds_const
20230125_083654


In [8]:
.61886* 63.81

39.4894566

In [9]:
.61886*60.58

37.490538799999996

In [10]:
(.61886* 63.81)/((26.2042+25.1558)/2)

1.5377514252336448

In [11]:
(.61886*60.58)/((23.7498+22.9137)/2)

1.6068464131494635

In [19]:
fringe_calibration1

(11150831,
 3.8056,
 0.5265,
 1.2824,
 array([], dtype=float64),
 array([], dtype=float64))

In [20]:
1.2824/.9

1.4248888888888889

In [13]:
calibration1=Calibrate('EPRData/01250837.dat',20221115083050,20221115083305,.6604,tempData)

Incorrect Grouping


IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
%%time
calibration1=Calibrate('EPRData/11150831.dat',20221115083050,20221115083305,.6604,tempData)

 Front shift is:  45.93 kHz 
 Back shift is:  43.92 kHz
 Format of output is: EPR date, pc_const, us_const, ds_const
20221115_083050
CPU times: user 4.39 s, sys: 0 ns, total: 4.39 s
Wall time: 4.39 s


In [ ]:
calibration1[1]

In [51]:
calibration1[2]

1.1544

In [52]:
calibration1[3]

1.8525

In [22]:
calibration1[4]

array([ 29.1371,  31.2116,  32.2249,  32.1285, 245.8489, 248.1074,
        31.1396,  62.4342])

In [53]:
calibration1[4]

array([ 30.1074,  32.8613,  22.0117,  33.2764,  17.5635,  33.0811,
        31.1572,  30.2051,  32.8809,  23.5107,  33.8525,  31.2402,
        32.8711,  33.8477,  33.7256,  25.1318,  27.3682,  33.7646,
        34.4922,  25.    ,  33.1641,  33.7891,  32.2705,  25.5762,
        34.7217,  34.5752,  29.3164,  35.127 ,  26.0693,  35.0342,
        34.8828,  30.    ,  35.0537,  29.9658,  34.1943,  34.1748,
        29.3213,  28.418 ,  34.1406,  34.0527,  26.6748,  34.0381,
        34.1211,  29.7168,  34.5605, 242.0092, 243.6525, 243.8314,
       245.5561, 245.2305, 248.2407, 246.5647, 247.5736, 247.6549,
       248.1754, 245.588 , 246.044 , 246.16  , 246.755 , 246.904 ,
       247.043 , 247.459 , 247.567 , 247.674 , 248.133 , 248.326 ,
       248.481 , 248.86  , 249.019 , 249.482 , 249.605 , 249.764 ,
       250.509 , 250.667 ,  30.7031,  31.5039,  30.8594,  31.2891,
        30.9082,  32.1484,  31.0254,  31.1719,  30.9277,  30.8594,
        68.999 ,  66.3672,  67.4072,  62.7881,  62.2705,  50.6

In [23]:
calibration1[5]

array([ 30.9195,  31.8483,  32.591 ,  32.2153, 248.4484, 250.7431,
        31.2378,  63.4292])

NMR_He_20221214_151106
12141512.dat
NMR_He_20221214_151329

In [8]:
s=.9

In [9]:
calibration2=Calibrate('EPRData/12141512.dat',20221214151106,20221214151329,.6604)

 Front shift is:  54.67 kHz 
 Back shift is:  52.44 kHz
 Format of output is: EPR date, pc_const, us_const, ds_const


In [10]:
calibration2[1]*s

6.697080000000001

In [11]:
calibration2[2]*s

2.18628

In [13]:
calibration2[3]*s

2.00475